# Langevin Dynamics of a free Particle


## Theory

- What is the Langevin equation? What is the dissipation force and what is friction or damping? What is the fluctuation force... how does the fluctuation force depend on temperature and friction?
- How does the fluctuation force effectively enter into the equations of motion to be integrated? Is the dimensionality of the term correct?
- What is a Brownian particle?
- What happens to a free particle at zero temperature and zero friction if it initially has non-zero velocity?
- What happens to a free particle at zero temperature and non-zero friction, if it initially has non-zero velocity?
- What happens to a free particle at nonzero temperature and zero friction, if it initially has a non-zero velocity?
- What happens to a free particle at nonzero temperature and nonzero friction, if it initially has non-zero velocity?
- What is the relationship between the average kinetic energy of a free particle and the temperature, and the kinetic energy given only by the X component of the velocity? Can you corroborate this relationship with a computational experiment? Do you think that this relationship could be used to calculate the temperature of the system by observing the average of the squared velocities of the particles of a system? Can you calculate analytically with statistical mechanics what is the expected kinetic energy of a free particle in a three-dimensional space at temperature T?
- What is the diffusion of a particle, what is its relation with the mean square dispersion of a particle, and with the temperature? Could you perform the appropriate computational experiments to corroborate the relation between the temperature of the system and the mean square distance that a particle has advanced from its initial position in a time t?
- What is the fluctuation-dissipation theorem?
- In the computational experiments you carried out, how did you choose the integration step? Is there any constraint to choose a small or large integration step?



In [ ]:
import openmm